# ConversationEntityMemory

`ConversationEntityMemory` cho phép hệ thống hội thoại giữ lại các sự kiện về các thực thể cụ thể được đề cập trong cuộc đối thoại.

Nó trích xuất thông tin về các thực thể từ cuộc trò chuyện (sử dụng LLM) và tích lũy kiến thức về các thực thể này theo thời gian (cũng sử dụng LLM).


In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Entity Memory Conversation Example

Ví dụ này minh họa cách sử dụng `ConversationEntityMemory` để lưu trữ và quản lý thông tin về các thực thể được đề cập trong một cuộc hội thoại. Cuộc hội thoại tích lũy kiến thức liên tục về các thực thể này trong khi vẫn duy trì luồng tự nhiên.


In [2]:
from langchain.chains import ConversationChain
from langchain.memory.entity import ConversationEntityMemory

In [3]:
from langchain.prompts import PromptTemplate

entity_memory_conversation_template = PromptTemplate(
    input_variables=["entities", "history", "input"],
    template="""
You are an assistant to a human, powered by a large language model trained by OpenAI.

You assist with various tasks, from answering simple questions to providing detailed discussions on a wide range of topics. You can generate human-like text, allowing natural conversations and coherent, relevant responses.

You constantly learn and improve, processing large amounts of text to provide accurate and informative responses. You can use personalized information provided in the context below, along with your own generated knowledge.

Context:
{entities}

Current conversation:
{history}
Last line:
Human: {input}
You:
""",
)

print(entity_memory_conversation_template)

input_variables=['entities', 'history', 'input'] input_types={} partial_variables={} template='\nYou are an assistant to a human, powered by a large language model trained by OpenAI.\n\nYou assist with various tasks, from answering simple questions to providing detailed discussions on a wide range of topics. You can generate human-like text, allowing natural conversations and coherent, relevant responses.\n\nYou constantly learn and improve, processing large amounts of text to provide accurate and informative responses. You can use personalized information provided in the context below, along with your own generated knowledge.\n\nContext:\n{entities}\n\nCurrent conversation:\n{history}\nLast line:\nHuman: {input}\nYou:\n'


In [4]:
conversation = ConversationChain(
    llm=llm,
    prompt=entity_memory_conversation_template,
    memory=ConversationEntityMemory(llm=llm),
)

/tmp/ipykernel_742366/4276868222.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationEntityMemory(llm=llm),
/home/dino/miniconda3/envs/langchain/lib/python3.10/site-packages/pydantic/main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/tmp/ipykernel_742366/4276868222.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [5]:
# Input conversation
response = conversation.predict(
    input=(
        "Amelia is an award-winning landscape photographer who has traveled around the globe capturing natural wonders. "
        "David is a wildlife conservationist dedicated to protecting endangered species. "
        "They are planning to open a nature-inspired photography gallery and learning center that raises funds for conservation projects."
    )
)

# Print the assistant's response
print(response)

That sounds like a wonderful initiative! Amelia's expertise in landscape photography combined with David's passion for wildlife conservation will surely create a unique and impactful gallery and learning center. By showcasing the beauty of nature through photography, they can raise awareness and support for conservation efforts. It's inspiring to see individuals using their talents to make a positive difference in the world.


## Retrieving Entity Memory
Let's examine the conversation history stored in memory using the `memory.entity_store.store` method to verify memory retention.

In [6]:
# Print the entity memory
conversation.memory.entity_store.store

{'Amelia': 'Amelia is an award-winning landscape photographer who has traveled around the globe capturing natural wonders.',
 'David': 'David is a wildlife conservationist dedicated to protecting endangered species.'}

### More infomation

**Khái niệm:**

`ConversationEntityMemory` là một loại bộ nhớ trong LangChain được thiết kế để theo dõi và lưu trữ các thực thể (entities) quan trọng được đề cập trong cuộc trò chuyện. Các thực thể này có thể là tên người, địa điểm, tổ chức, hoặc bất kỳ đối tượng nào khác mà mô hình ngôn ngữ cho là quan trọng để duy trì ngữ cảnh của cuộc hội thoại.

Không giống như các loại bộ nhớ đơn giản chỉ lưu trữ toàn bộ lịch sử trò chuyện, `ConversationEntityMemory` tập trung vào việc trích xuất và lưu trữ các thông tin cốt lõi về các thực thể này. Điều này giúp mô hình ngôn ngữ hiểu rõ hơn về những gì đã được thảo luận và đưa ra phản hồi chính xác hơn.

**Ưu điểm:**

* **Hiểu ngữ cảnh sâu sắc hơn:** Bằng cách theo dõi các thực thể, mô hình có thể duy trì ngữ cảnh tốt hơn, ngay cả khi cuộc trò chuyện kéo dài.
* **Giảm thiểu thông tin thừa:** Chỉ lưu trữ các thực thể quan trọng giúp tiết kiệm tài nguyên và tăng hiệu quả xử lý.
* **Phản hồi chính xác hơn:** Khả năng nhận biết và ghi nhớ các thực thể giúp mô hình đưa ra phản hồi phù hợp và chính xác hơn.

**Hướng dẫn sử dụng:**

1.  **Cài đặt:**
    
    ```bash
    pip install langchain
    ```
    
2.  **Khởi tạo:**
    
    ```python
    from langchain.memory import ConversationEntityMemory
    from langchain.llms import OpenAI
    
    llm = OpenAI(temperature=0)
    memory = ConversationEntityMemory(llm=llm)
    ```
    
3.  **Sử dụng trong chuỗi hội thoại:**
    
    ```python
    memory.save_context({"input": "Xin chào, tên tôi là An."}, {"output": "Chào An!"})
    memory.save_context({"input": "Tôi sống ở Hà Nội."}, {"output": "Hà Nội là một thành phố tuyệt vời!"})
    
    print(memory.load_memory_variables({}))
    ```
    
    Trong ví dụ này:
    
    * `memory.save_context()` được sử dụng để lưu trữ các lượt hội thoại.
    * `memory.load_memory_variables({})` được sử dụng để truy xuất các thực thể đã được lưu trữ.
    

**Ví dụ minh họa:**

```python
from langchain.memory import ConversationEntityMemory
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)
memory = ConversationEntityMemory(llm=llm)

template = """Đây là cuộc hội thoại trước đó:
{history}
Bây giờ, hãy trả lời câu hỏi sau: {input}"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = LLMChain(llm=llm, prompt=prompt, memory=memory)

print(conversation.predict(input="Tên của tôi là An."))
print(conversation.predict(input="Tôi sống ở đâu?"))
```

Trong ví dụ này:

* `ConversationEntityMemory` giúp mô hình ghi nhớ tên "An" và địa điểm "Hà Nội".
* Khi được hỏi "Tôi sống ở đâu?", mô hình có thể truy xuất thông tin từ bộ nhớ và trả lời chính xác.

**Lưu ý:**

* `ConversationEntityMemory` phụ thuộc vào khả năng trích xuất thực thể của mô hình ngôn ngữ.
* Hiệu quả của nó có thể thay đổi tùy thuộc vào độ phức tạp của cuộc trò chuyện và chất lượng của mô hình ngôn ngữ.

